# Лабораторна робота №6 - ФБ-31 Гек
## Застосування numpy

**Мета роботи:** отримати поглиблені навички роботи з numpy; дослідити поняття лінійної регресії та градієнтного спуску.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Завдання 1.1 Генерація прямої та шуму навколо неї

In [ ]:
# Коеф. прямої
k = -0.7  # нахил прямої
b = 2     # перетин з віссю y
N = 500   # к-ть точок

# Генеруємо значення x
x = np.linspace(0, 10, N)

# Генеруємо значення y
noise = np.random.normal(0, 1.5, N)  # шум з нормальним розподілом навколо прямої
y = k * x + b + noise

# Відображення
plt.figure(figsize=(10, 6))
plt.scatter(x, y, label='Дані', color='purple', alpha=0.6)
plt.plot(x, k * x + b, color='red', label='Пряма')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Дані з шумом та справжня пряма')
plt.legend()
plt.grid(True)
plt.show()

## Завдання 1.(2-4) Метод найменших квадратів


$$b=\frac{\sum_{i=1}^{n}(x_i^2)\sum_{i=1}^{n}(y_i) - \sum_{i=1}^{n}(x_i)\sum_{i=1}^{n}(x_i y_i)}{n\sum_{i=1}^{n}(x_i^2)-(\sum_{i=1}^{n}(x_i))^2}$$

$$k=\frac{n\sum_{i=1}^{n}(x_i y_i)-(\sum_{i=1}^{n}(x_i))(\sum_{i=1}^{n}(y_i))}{n\sum_{i=1}^{n}(x_i^2)-(\sum_{i=1}^{n}(x_i))^2}$$

In [ ]:
# Метод найменших квадратів
def least_squares_method(x, y):
    n = len(x)
    sum_x = np.sum(x)
    sum_y = np.sum(y)
    sum_xy = np.sum(x * y)
    sum_xx = np.sum(x * x)
    
    # Коеф. прямої
    kk_hat = (n * sum_xy - sum_x * sum_y) / (n * sum_xx - sum_x ** 2)
    bb_hat = (sum_y - kk_hat * sum_x) / n
    
    return kk_hat, bb_hat

# Оцінки коеф. за методом найменших квадратів
kk_hat, bb_hat = least_squares_method(x, y)

# Оцінки коеф. за допомогою np.polyfit
coefficients = np.polyfit(x, y, 1)
kk_np_polyfit = coefficients[0]
bb_np_polyfit = coefficients[1]

plt.figure(figsize=(10, 6))
plt.scatter(x, y, label='Дані', color='purple', alpha=0.6)
plt.plot(x, k * x + b, color='red', label='Пряма')
plt.plot(x, kk_hat * x + bb_hat, color='green', linestyle='--', label='Оцінка методом найменших квадратів')
plt.plot(x, kk_np_polyfit * x + bb_np_polyfit, color='blue', linestyle='-.', label='Оцінка np.polyfit')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Оцінки методом найменших квадратів і np.polyfit')
plt.legend()
plt.grid(True)
plt.show()

print("Парам. справжньої прямої:")
print("k =", k)
print("b =", b)
print()
print("Оцінки парам. за методом найменших квадратів:")
print("kk_hat =", kk_hat)
print("bb_hat =", bb_hat)
print()
print("Оцінки парам. за np.polyfit:")
print("kk_np_polyfit =", kk_np_polyfit)
print("bb_np_polyfit =", bb_np_polyfit)

## Завдання 2. Метод градієнтного спуску

Метод градієнтного спуску використовує ітеративний підхід для знаходження мінімуму ф-ї втрат (у нашому випадку - середньоквадратичної помилки). Алгоритм починає з певних початкових значень парам. і поступово коригує їх, рухаючись у напрямку антиградієнта ф-ї втрат.

Часткові похідні ф-ї втрат:

$$\frac{\partial L}{\partial k} = -\frac{2}{n} \sum_{i=1}^{n} x_i (y_i - (k x_i + b))$$

$$\frac{\partial L}{\partial b} = -\frac{2}{n} \sum_{i=1}^{n} (y_i - (k x_i + b))$$


In [ ]:
def gradient_descent(x, y, learning_rate, n_iter):

    # Початкові оцінки коеф.
    kk_hat = 0.0
    bb_hat = 0.0
    n = len(x)
    mse_history = []
    
    # Ітеративний процес градієнтного спуску
    for i in range(n_iter):
        # Передбачені значення y
        y_pred = kk_hat * x + bb_hat
        
        # Обчислення часткових похідних (градієнтів)
        gradient_kk = (-2/n) * np.sum(x * (y - y_pred))
        gradient_bb = (-2/n) * np.sum(y - y_pred)
        
        # Оновлення парам.
        kk_hat = kk_hat - learning_rate * gradient_kk
        bb_hat = bb_hat - learning_rate * gradient_bb
        
        # Обчислення середньоквадратичної помилки (MSE)
        mse = np.mean((y - y_pred) ** 2)
        mse_history.append(mse)
    
    return kk_hat, bb_hat, mse_history

# Визначення оптимальних парам. градієнтного спуску
learning_rate = 0.01
n_iter = 1000

# Виклик ф-ї градієнтного спуску
kk_gd, bb_gd, mse_history = gradient_descent(x, y, learning_rate, n_iter)


plt.figure(figsize=(10, 6))
plt.scatter(x, y, label='Дані', color='purple', alpha=0.6)
plt.plot(x, k * x + b, color='red', label='Пряма')
plt.plot(x, kk_hat * x + bb_hat, color='blue', linestyle='--', label='Оцінка методом найменших квадратів')
plt.plot(x, kk_np_polyfit * x + bb_np_polyfit, color='green', linestyle='-.', label='Оцінка за np.polyfit')
plt.plot(x, kk_gd * x + bb_gd, color='orange', linestyle=':', label='Оцінка градієнтним спуском')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Порівняння методів регресійного аналізу')
plt.legend()
plt.grid(True)
plt.show()

print("Парам. справжньої прямої:")
print("k =", k)
print("b =", b)
print()
print("Оцінки парам. за методом найменших квадратів:")
print("kk_hat =", kk_hat)
print("bb_hat =", bb_hat)
print()
print("Оцінки парам. за градієнтним спуском:")
print("kk_gd =", kk_gd)
print("bb_gd =", bb_gd)

## Дослідження впливу парам. градієнтного спуску

In [ ]:
# Побудова графіка похибки в залежності від к-ті ітерацій
plt.figure(figsize=(10, 6))
plt.plot(range(1, n_iter + 1), mse_history, color='blue')
plt.xlabel('к-ть ітерацій')
plt.ylabel('Середньоквадратична похибка (MSE)')
plt.title('Залежність похибки від к-ті ітерацій')
plt.grid(True)
plt.show()

In [ ]:
# Вплив швидкості навчання на збіжність градієнтного спуску
learning_rates = [0.001, 0.01, 0.05, 0.1]
n_iter = 1000

plt.figure(figsize=(12, 8))

for lr in learning_rates:
    kk_gd, bb_gd, mse_history = gradient_descent(x, y, lr, n_iter)
    plt.plot(range(1, n_iter + 1), mse_history, label=f'learning_rate = {lr}')
    
plt.xlabel('к-ть ітерацій')
plt.ylabel('Середньоквадратична похибка (MSE)')
plt.title('Вплив швидкості навчання на збіжність градієнтного спуску')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Порівняємо кінцеві результати для різних швидкостей навчання
learning_rates = [0.001, 0.01, 0.05, 0.1]
n_iter = 1000

plt.figure(figsize=(10, 6))
plt.scatter(x, y, label='Дані', color='purple', alpha=0.3)
plt.plot(x, k * x + b, color='red', label='Пряма')

# Додаємо результати метода найменших квадратів для порівняння
plt.plot(x, kk_hat * x + bb_hat, color='green', linestyle='--', label='Метод найменших квадратів')

for lr in learning_rates:
    kk_gd, bb_gd, _ = gradient_descent(x, y, lr, n_iter)
    plt.plot(x, kk_gd * x + bb_gd, linestyle=':', label=f'GD, lr = {lr}')
    
plt.xlabel('x')
plt.ylabel('y')
plt.title('Результати градієнтного спуску з різними швидкостями навчання')
plt.legend()
plt.grid(True)
plt.show()

## Висновки

У цій лабораторній роботі реалізував два методи для знаходження парам. лінійної регресії:
1. **Метод найменших квадратів (МНК)** - точне рішення за одну ітерацію
2. **Метод градієнтного спуску** - ітеративний метод, який поступово наближається до оптимального рішення

Порівняв ці методи та дослідили вплив різних парам. градієнтного спуску (швидкості навчання та к-ті ітерацій) на точність отриманих результатів.

Метод найменших квадратів дає точне рішення за одну ітерацію, що є його перевагою для простих лінійних моделей
Градієнтний спуск потребує налаштування гіперпарам. і більшої к-ті обчислень, але є більш універсальним для складніших моделей
Занадто велика швидкість навчання може призвести до розбіжності алгоритму
Занадто мала швидкість навчання вимагає більшої к-ті ітерацій для досягнення оптимального результату

У моєму випадку обидва методи дають дуже схожі результати, що підтверджує коректність їх реалізації та ефективність для задачі лінійної регресії.